In [ ]:


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os
import csv
from scipy.stats import f_oneway
from scipy.stats import shapiro, levene, probplot
import math
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd

#conversion factors
in_to_m = 0.0254

def load_tensile_csv(csv_path):
    """
    Loads tensile raw data CSVs that may contain:
      - metadata header lines
      - mixed delimiters
      - inconsistent column counts
    Returns a cleaned DataFrame with numeric columns only.
    """

    # First: Detect delimiter
    with open(csv_path, "r") as f:
        sample = f.read(2048)

    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample, delimiters=[',',';','\t',' '])
        delimiter = dialect.delimiter
    except:
        delimiter = ','  # fallback

    # Second: Read the file while skipping bad rows
    df = pd.read_csv(
        csv_path,
        sep=delimiter,
        engine="python",
        comment='#',
        skiprows=5,
        #on_bad_lines='skip',    # <-- prevents crash
        skip_blank_lines=True
    )

    # Third: Drop EMPTY or non-numeric columns
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            df[col] = pd.NA

    df = df.dropna(axis=1, how='all')  # remove columns with no data
    df = df.dropna(axis=0, how='any')  # remove unfinished rows

    print(df)
    return df

def compute_youngs_modulus(
    csv_path,
    gauge_length,
    cross_section_area,
    load_col='Load',
    disp_col='Extension',
    elastic_strain_max=0.005
):
    """
    Computes Young's modulus from tensile test data.

    Parameters
    ----------
    csv_path : str
        Path to CSV file containing load/displacement data.
    gauge_length : float
        Original specimen length (m).
    cross_section_area : float
        Cross-sectional area (m^2).
    load_col : str
        Column name for load/force.
    disp_col : str
        Column name for displacement/extension.
    elastic_strain_max : float
        Maximum strain to use for the linear elastic fit (default = 0.5%).

    Returns
    -------
    E : float
        Estimated Young’s modulus (Pa).
    stress : ndarray
    strain : ndarray
        Arrays of engineering stress and strain.
    """

    # Load file
    df = load_tensile_csv(csv_path)

    # Extract data
    force = df[load_col].values
    disp  = df[disp_col].values

    # Engineering strain
    strain = disp / gauge_length

    # Engineering stress
    stress = force / cross_section_area

    # Select elastic region
    mask = strain <= elastic_strain_max
    strain_elastic = strain[mask].reshape(-1, 1)
    stress_elastic = stress[mask]

    # Linear regression (slope = E)
    model = LinearRegression().fit(strain_elastic, stress_elastic)
    E = model.coef_[0]

    return E, stress, strain


In [2]:
def ultimate_tensile_strength(stress):
    """
    Maximum stress achieved in the test.
    """
    return np.max(stress)

def offset_yield_strength(E, strain, stress, offset=0.02):
    """
    Computes the offset yield strength (default 2%).
    Uses interpolation to find intersection of stress-strain curve with
    offset line of slope equal to the elastic modulus.
    """
    offset_line = E*(strain - offset)

    # difference between actual curve and offset line
    diff = stress - offset_line
    
    # find zero crossing
    idx = np.where(np.diff(np.sign(diff)))[0]
    if len(idx) == 0:
        return None  # No yield point found
    
    i = idx[0]
    # linear interpolation for better accuracy
    x1, x2 = strain[i], strain[i+1]
    y1, y2 = diff[i], diff[i+1]
    
    strain_yield = x1 - y1*(x2 - x1)/(y2 - y1)
    stress_yield = np.interp(strain_yield, strain, stress)
    
    return stress_yield

def total_energy(strain, stress):
    """
    Integrates the stress-strain curve using the trapezoidal rule.
    Units depend on stress/strain units (e.g., MPa -> MJ/m^3).
    """
    return np.trapz(stress, strain)

def strain_at_fracture(strain):
    """
    Maximum strain recorded before failure.
    """
    return np.max(strain)

In [3]:
def analyze_tensile_folder(folder_path, gauge_length, cross_section_area):
    """
    Processes all CSV tensile data files in a folder and returns a summary table.
    """

    results = []

    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    # print(csv_files)

    for csv_path in csv_files:
        try:
            # --- Stress & strain + Young's modulus ---
            E, stress, strain = compute_youngs_modulus(
                csv_path,
                gauge_length,
                cross_section_area
            )
            # print("fail 1")

            # --- Individual metrics ---
            uts = ultimate_tensile_strength(stress)
            # print("fail1.5")
            #ys  = offset_yield_strength(strain, stress, E)
            # print("fail1.5")
            energy = total_energy(strain, stress)
            # print("fail1.5")

            eps_f = strain_at_fracture(strain)
            # print("fail 2")
            results.append({
                "Filename": os.path.basename(csv_path),
                "Youngs_Modulus_Pa": E,
                "UTS_Pa": uts,
            #    "YieldStrength_2pct_Pa": ys,
                "TotalEnergy_Jperm3": energy,
                "StrainAtFracture": eps_f
            })

            # print(results, uts, energy, eps_f)

        except Exception as e:
            print(f"Error processing {csv_path}: {e}")

    return pd.DataFrame(results)


In [4]:
reference_tests = analyze_tensile_folder(
     "./Reference Tests/Reference_T1",
    1, 1
)

print(reference_tests)


       Time  Extension         Load
1     0.000   -0.04256      1.60519
2     0.100   -0.03807      1.36151
3     0.200   -0.02903     12.88276
4     0.300   -0.01960    149.19690
5     0.400   -0.01041    256.47833
..      ...        ...          ...
717  71.062    5.87952  11136.10156
718  71.064    5.87972  10348.32031
719  71.066    5.87992   9540.51465
720  71.068    5.88010   8732.13672
721  71.070    5.88030   7930.69141

[721 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000      2.46954
2     0.100    0.00403      4.91734
3     0.200    0.01346     88.88831
4     0.300    0.02311    184.77623
5     0.400    0.03238    305.11151
..      ...        ...          ...
739  73.342    6.11218  10917.00684
740  73.344    6.11238  10172.36230
741  73.346    6.11258   9401.66895
742  73.348    6.11278   8625.38672
743  73.350    6.11298   7852.75635

[743 rows x 3 columns]
       Time  Extension         Load
1     0.000   -0.08265     13.19987
2     0.100   -0

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

In [5]:
#ANOVA TEST
def run_anova_on_metric(results_dict, metric_col):
    """
    results_dict: dict[name -> DataFrame from analyze_tensile_folder]
    metric_col: string name of the column to test, e.g. "UTS_Pa"
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_dict.items():
        # pull that column, drop NaN
        vals = df[metric_col].dropna().values

        # we need at least 2 samples for ANOVA to make sense
        if len(vals) >= 2:
            group_values.append(vals)
            group_labels.append(condition_name)
        else:
            print(f"Skipping {condition_name} for {metric_col} (only {len(vals)} sample(s))")

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for ANOVA on {metric_col}")

    F, p = f_oneway(*group_values)

    print(f"\nANOVA for {metric_col}")
    print("Groups:", group_labels)
    print(f"F-statistic = {F:.4f}, p-value = {p:.4e}")

    return F, p, group_labels

In [6]:
#Shapiro test
def run_shapiro_per_group(results_by_condition, metric_col):
    """
    Run Shapiro–Wilk test for normality in each group (condition) for one metric.
    """
    print(f"\n=== Shapiro–Wilk normality test for {metric_col} ===")
    shapiro_results = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values

        # Shapiro needs at least 3 points to be meaningful
        if len(vals) < 3:
            print(f"{condition_name}: n={len(vals)} (skipping, too few samples)")
            continue

        stat, p = shapiro(vals)
        shapiro_results.append({
            "Condition": condition_name,
            "n": len(vals),
            "W_stat": stat,
            "p_value": p
        })

        interpretation = "✅ approx. normal (fail to reject H0)" if p > 0.05 else "⚠️ non-normal (reject H0)"
        print(f"{condition_name}: n={len(vals)}, W={stat:.4f}, p={p:.4f} → {interpretation}")

    return pd.DataFrame(shapiro_results)


In [7]:
#Q-Q plots

def qq_plots_per_group(results_by_condition, metric_col):
    """
    Generate Q–Q plots for each group (condition) for one metric.
    """
    n_groups = len(results_by_condition)
    ncols = 3  # adjust if you want a different layout
    nrows = math.ceil(n_groups / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows))
    axes = np.array(axes).reshape(-1)  # flatten in case of 2D

    fig.suptitle(f"Q–Q plots for {metric_col}", fontsize=16)

    for ax, (condition_name, df) in zip(axes, results_by_condition.items()):
        vals = df[metric_col].dropna().values

        if len(vals) < 3:
            ax.set_title(f"{condition_name}\n(n={len(vals)}, skipped)")
            ax.axis("off")
            continue

        probplot(vals, dist="norm", plot=ax)
        ax.set_title(f"{condition_name} (n={len(vals)})")

    # Hide any extra axes if number of groups < nrows*ncols
    for j in range(len(results_by_condition), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


In [8]:
#LEVENES test
def run_levene_across_groups(results_by_condition, metric_col, center='median'):
    """
    Run Levene’s test for homogeneity of variances across groups for one metric.
    center: 'median' (more robust) or 'mean'
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) < 2:
            print(f"{condition_name}: n={len(vals)} (skipping in Levene, too few samples)")
            continue
        group_values.append(vals)
        group_labels.append(condition_name)

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for Levene’s test on {metric_col}")

    stat, p = levene(*group_values, center=center)

    interpretation = "✅ variances approx. equal (fail to reject H0)" if p > 0.05 else "⚠️ variances differ (reject H0)"

    print(f"\n=== Levene’s test for {metric_col} (center={center}) ===")
    print("Groups:", group_labels)
    print(f"Levene stat = {stat:.4f}, p = {p:.4f} → {interpretation}")

    return stat, p, group_labels


In [9]:
#Tukey HSD
def run_tukey_for_metric(results_by_condition, metric_col, alpha=0.05):
    """
    results_by_condition: dict[condition_name -> DataFrame from analyze_tensile_folder]
    metric_col: e.g. "UTS_Pa", "TotalEnergy_Jperm3", "StrainAtFracture"
    """
    all_values = []
    all_groups = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) == 0:
            continue

        all_values.extend(vals)
        all_groups.extend([condition_name] * len(vals))

    all_values = np.array(all_values)
    all_groups = np.array(all_groups)

    tukey = pairwise_tukeyhsd(
        endog=all_values,   # the data
        groups=all_groups,  # group labels (folder names)
        alpha=alpha
    )

    print(f"\n=== Tukey HSD for {metric_col} (alpha = {alpha}) ===")
    print(tukey.summary())   # nice text table

    # Convert to DataFrame for easier viewing / export
    data = tukey._results_table.data
    header, rows = data[0], data[1:]
    tukey_df = pd.DataFrame(rows, columns=header)

    return tukey, tukey_df


In [ ]:
#After Sinter Part Dimensino Dictionary
after_print_dim = pd.read_csv("after_sinter_dimensions.csv")
all_combinations = ["Gyroid_8", "Gyroid_6", "Gyroid_4", "Solid_8", "Solid_6", "Solid_4", "Triangular_8", "Triangular_6", "Triangular_4"]

measurement_types = ["length (in)", "thickness (in)", "width (in)", "mass (g)"]

combo_to_measurement = {}
for combination in all_combinations:
    sub_df = after_print_dim[(after_print_dim["infill_type"] == combination.split("_")[0].lower()) & (after_print_dim["wall_thickness"] == float(combination.split("_")[1]))]
    
    length = float(sub_df[sub_df["measurement"] == measurement_types[0]]["mean_of_part_means"].values[0]) * in_to_m

    cross_section_area = float(sub_df[sub_df["measurement"] == measurement_types[1]]["mean_of_part_means"].values[0]) * float(sub_df[sub_df["measurement"] == measurement_types[2]]["mean_of_part_means"].values[0])*in_to_m**2

    mean =  float(sub_df[sub_df["measurement"] == measurement_types[3]]["mean_of_part_means"].values[0])
    combo_to_measurement[combination] = [length, cross_section_area,mean]



{'Gyroid_8': [0.129809875, 2.968490299566666e-05, 18.78], 'Gyroid_6': [0.12990321999999999, 3.037943789691664e-05, 17.19], 'Gyroid_4': [0.12987527999999998, 3.018365468633333e-05, 15.7875], 'Solid_8': [0.130062605, 2.9996963527458326e-05, 24.33], 'Solid_6': [0.13011975499999998, 2.9683725578666672e-05, 24.2705], 'Solid_4': [0.13000609, 2.873698553666667e-05, 24.25], 'Triangular_8': [0.12988163, 3.0446595692708332e-05, 18.728], 'Triangular_6': [0.12980416, 3.009802851349999e-05, 17.5], 'Triangular_4': [0.12973240500000002, 2.955536024399997e-05, 15.966]}


In [27]:
#Goal 1

#get relevant folers
folder_names = ["Gyroid_8", "Gyroid_6", "Gyroid_4", "Solid_8", "Solid_6", "Solid_4", "Triangular_8", "Triangular_6", "Triangular_4"]
folder_addendum = ".is_tens_RawData"
parent_dir = "3DPrint"
folder_names_final = [os.path.join(parent_dir, name + folder_addendum) for name in folder_names]

#define lengths and section areas
L = 1*in_to_m #in
A = 0.125*0.25*in_to_m**2 #in^2

#
actual_length = 4.349*in_to_m
actual_area = (0.125*0.25)*in_to_m**2

#getting all the references
results_by_condition = {}

for i, folder_name in enumerate(folder_names_final):

    reference_tests = analyze_tensile_folder(
        folder_name,
        combo_to_measurement[folder_names[i]][0]/actual_length*L, combo_to_measurement[folder_names[i]][1]/actual_area*A
    )

    results_by_condition[folder_name] = reference_tests

#define metrics
metrics = [
    "UTS_Pa",              # ultimate tensile strength
    "TotalEnergy_Jperm3",  # energy absorbed
    "StrainAtFracture"     # elongation at break
]

#ANOVA
anova_results = []

for metric in metrics:
    F, p, groups = run_anova_on_metric(results_by_condition, metric)
    anova_results.append({
        "Metric": metric,
        "F_stat": F,
        "p_value": p
    })

print(anova_results)

#Shaprio
for metric in metrics:
    shapiro_df = run_shapiro_per_group(results_by_condition, metric)
    display(shapiro_df)

#levene's
for metric in metrics:
    run_levene_across_groups(results_by_condition, metric, center='median')

#Q-Q PLOT
# for metric in metrics:
#     qq_plots_per_group(results_by_condition, metric)

#Tukey
tukey_tables = {}

for metric in metrics:
    tukey_result, tukey_df = run_tukey_for_metric(results_by_condition, metric)
    tukey_tables[metric] = tukey_df
    display(tukey_df)



       Time  Extension         Load
1     0.000    0.00030     -2.64829
2     0.100    0.00764     43.94266
3     0.200    0.01751    104.44289
4     0.300    0.02691    132.00453
5     0.400    0.03581    182.97757
..      ...        ...          ...
384  37.884    3.16041  14899.19434
385  37.886    3.16064  13925.96191
386  37.888    3.16084  12905.07715
387  37.890    3.16104  11863.46484
388  37.892    3.16127  10824.25000

[388 rows x 3 columns]
Error processing 3DPrint/Gyroid_8.is_tens_RawData/Specimen_RawData_4.csv: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by LinearRegression.
       Time  Extension         Load
1     0.000    0.00000      0.93332
2     0.100    0.00403      2.96457
3     0.200    0.01341      7.14769
4     0.300    0.02283     80.13934
5     0.400    0.03210    169.43657
..      ...        ...          ...
364  35.826    2.98581  14494.72461
365  35.828    2.98601  13531.48340
366  35.830    2.98621  12524.41406
367  35.832 

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,3,0.834635,0.200180
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.792358,0.089206
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.950580,0.719740
3,3DPrint/Solid_8.is_tens_RawData,3,0.853814,0.250705
4,3DPrint/Solid_6.is_tens_RawData,4,0.951657,0.726480
5,3DPrint/Solid_4.is_tens_RawData,4,0.824975,0.155074
6,3DPrint/Triangular_8.is_tens_RawData,4,0.911332,0.489485
7,3DPrint/Triangular_6.is_tens_RawData,4,0.999425,0.998424
8,3DPrint/Triangular_4.is_tens_RawData,4,0.934332,0.620089



=== Shapiro–Wilk normality test for TotalEnergy_Jperm3 ===
3DPrint/Gyroid_8.is_tens_RawData: n=3, W=0.9914, p=0.8224 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_6.is_tens_RawData: n=4, W=0.8827, p=0.3502 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_4.is_tens_RawData: n=4, W=0.9702, p=0.8427 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_8.is_tens_RawData: n=3, W=0.9867, p=0.7792 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_6.is_tens_RawData: n=4, W=0.9129, p=0.4978 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_4.is_tens_RawData: n=4, W=0.8315, p=0.1718 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_8.is_tens_RawData: n=4, W=0.8570, p=0.2497 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_6.is_tens_RawData: n=4, W=0.9842, p=0.9261 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_4.is_tens_RawData: n=4, W=0.9535, p=0.7378 → ✅ approx. normal (fail to reject H0)


,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,3,0.991373,0.822355
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.882681,0.350212
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.970195,0.842693
3,3DPrint/Solid_8.is_tens_RawData,3,0.986689,0.779158
4,3DPrint/Solid_6.is_tens_RawData,4,0.912882,0.497804
5,3DPrint/Solid_4.is_tens_RawData,4,0.831538,0.171849
6,3DPrint/Triangular_8.is_tens_RawData,4,0.857010,0.249679
7,3DPrint/Triangular_6.is_tens_RawData,4,0.984180,0.926068
8,3DPrint/Triangular_4.is_tens_RawData,4,0.953459,0.737779


...

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,8.507103e+07,0.0000,7.612774e+07,9.401433e+07,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,1.470710e+08,0.0000,1.374111e+08,1.567308e+08,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,4.097789e+08,0.0000,4.008356e+08,4.187222e+08,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,3.908853e+08,0.0000,3.819420e+08,3.998286e+08,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,3.825517e+08,0.0000,3.728918e+08,3.922115e+08,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,1.774557e+06,0.9988,-7.168735e+06,1.071785e+07,False
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,6.289513e+07,0.0000,5.395184e+07,7.183843e+07,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,1.427780e+08,0.0000,1.338347e+08,1.517213e+08,True
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,6.199993e+07,0.0000,5.234007e+07,7.165979e+07,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,3.247079e+08,0.0000,3.157646e+08,3.336511e+08,True



=== Tukey HSD for TotalEnergy_Jperm3 (alpha = 0.05) ===
                                             Multiple Comparison of Means - Tukey HSD, FWER=0.05                                             
               group1                               group2                     meandiff     p-adj        lower             upper       reject
---------------------------------------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData  10270610616.7246    0.0   4946046573.8426  15595174659.6065   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData   6498955127.6032 0.0184    747768645.2725  12250141609.9339   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData  55646806574.7085    0.0  50322242531.8265  60971370617.5904   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData  55563748486.7561

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,1.027061e+10,0.0000,4.946047e+09,1.559517e+10,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,6.498955e+09,0.0184,7.477686e+08,1.225014e+10,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,5.564681e+10,0.0000,5.032224e+10,6.097137e+10,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,5.556375e+10,0.0000,5.023918e+10,6.088831e+10,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,5.400971e+10,0.0000,4.825853e+10,5.976090e+10,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-6.703792e+09,0.0065,-1.202836e+10,-1.379228e+09,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-1.287804e+09,0.9951,-6.612368e+09,4.036760e+09,False
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,4.784491e+09,0.1030,-5.400729e+08,1.010906e+10,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-3.771655e+09,0.4231,-9.522842e+09,1.979531e+09,False
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,4.537620e+10,0.0000,4.005163e+10,5.070076e+10,True



=== Tukey HSD for StrainAtFracture (alpha = 0.05) ===
                               Multiple Comparison of Means - Tukey HSD, FWER=0.05                                
               group1                               group2                meandiff p-adj   lower    upper   reject
------------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData  15.1209 0.0143   2.1028  28.1391   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData  -1.2468    1.0  -15.308  12.8144  False
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData   49.413    0.0  36.3949  62.4312   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData  47.7634    0.0  34.7452  60.7816   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_8.is_tens_RawData  47.8596    0.0  33.7984  61.9208   True
    3DPrint/Gyroid_4.is_t

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,15.1209,0.0143,2.1028,28.1391,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-1.2468,1.0000,-15.3080,12.8144,False
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,49.4130,0.0000,36.3949,62.4312,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,47.7634,0.0000,34.7452,60.7816,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,47.8596,0.0000,33.7984,61.9208,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-23.5385,0.0001,-36.5566,-10.5203,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-14.0753,0.0270,-27.0935,-1.0571,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,-7.0587,0.6595,-20.0768,5.9595,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-16.3677,0.0141,-30.4290,-2.3065,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,34.2921,0.0000,21.2739,47.3103,True


In [28]:
#Goal 4

#get relevant folers
folder_names = [ "Solid_8", "Solid_6", "Solid_4"]
folder_addendum = ".is_tens_RawData"
parent_dir = "3DPrint"
folder_names_final = [os.path.join(parent_dir, name + folder_addendum) for name in folder_names] 
folder_names_final = folder_names_final + ["./Reference Tests/Reference_T1"]

#define lengths and section areas
L = 1*in_to_m #in
A = 0.125*0.25*in_to_m**2 #in^2

actual_length = 4.349*in_to_m
actual_area = (0.125*0.25)*in_to_m**2

#getting all the references
results_by_condition = {}

for i, folder_name in enumerate(folder_names_final):

    #use specific dimensions
    if not "Reference" in folder_name: 
        reference_tests = analyze_tensile_folder(
            folder_name,
        combo_to_measurement[folder_names[i]][0]/actual_length*L, combo_to_measurement[folder_names[i]][1]/actual_area*A
        )
    #USE CAD DIMENSIONS (for waterjet parts)
    else:
        reference_tests = analyze_tensile_folder(
            folder_name,
            L, A)
        
    results_by_condition[folder_name] = reference_tests

#define metrics
metrics = [
    "UTS_Pa",              # ultimate tensile strength
    "TotalEnergy_Jperm3",  # energy absorbed
    "StrainAtFracture"     # elongation at break
]

#ANOVA
anova_results = []

for metric in metrics:
    F, p, groups = run_anova_on_metric(results_by_condition, metric)
    anova_results.append({
        "Metric": metric,
        "F_stat": F,
        "p_value": p
    })

print(anova_results)

#Shaprio
for metric in metrics:
    shapiro_df = run_shapiro_per_group(results_by_condition, metric)
    display(shapiro_df)

#levene's
for metric in metrics:
    run_levene_across_groups(results_by_condition, metric, center='median')

#Q-Q PLOT
# for metric in metrics:
#     qq_plots_per_group(results_by_condition, metric)

#Tukey
tukey_tables = {}

for metric in metrics:
    tukey_result, tukey_df = run_tukey_for_metric(results_by_condition, metric)
    tukey_tables[metric] = tukey_df
    display(tukey_df)



       Time  Extension         Load
1     0.000    0.00000      3.47343
2     0.100    0.00456      4.32416
3     0.200    0.01381     10.19229
4     0.300    0.02301     57.88463
5     0.400    0.03223    138.46074
..      ...        ...          ...
511  50.572    4.21466  20361.54297
512  50.574    4.21489  19005.10938
513  50.576    4.21511  17587.25586
514  50.578    4.21532  16151.84570
515  50.580    4.21557  14723.18164

[515 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000     39.61577
2     0.100    0.00428     39.44108
3     0.200    0.01358     45.92258
4     0.300    0.02291    121.67086
5     0.400    0.03218    232.67726
..      ...        ...          ...
591  58.536    4.87829  20049.78125
592  58.538    4.87851  18733.70508
593  58.540    4.87874  17351.25586
594  58.542    4.87897  15942.98633
595  58.544    4.87922  14539.67871

[595 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000      4.78062
2     0.100    0

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_86233/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

,Condition,n,W_stat,p_value
0,3DPrint/Solid_8.is_tens_RawData,3,0.853814,0.250705
1,3DPrint/Solid_6.is_tens_RawData,4,0.951657,0.726480
2,3DPrint/Solid_4.is_tens_RawData,4,0.824975,0.155074
3,./Reference Tests/Reference_T1,9,0.985360,0.986112



=== Shapiro–Wilk normality test for TotalEnergy_Jperm3 ===
3DPrint/Solid_8.is_tens_RawData: n=3, W=0.9867, p=0.7792 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_6.is_tens_RawData: n=4, W=0.9129, p=0.4978 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_4.is_tens_RawData: n=4, W=0.8315, p=0.1718 → ✅ approx. normal (fail to reject H0)
./Reference Tests/Reference_T1: n=9, W=0.9184, p=0.3789 → ✅ approx. normal (fail to reject H0)


,Condition,n,W_stat,p_value
0,3DPrint/Solid_8.is_tens_RawData,3,0.986689,0.779158
1,3DPrint/Solid_6.is_tens_RawData,4,0.912882,0.497804
2,3DPrint/Solid_4.is_tens_RawData,4,0.831538,0.171849
3,./Reference Tests/Reference_T1,9,0.918364,0.378866


...

,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,-2.825432e+08,0.0,-2.845853e+08,-2.805011e+08,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,-3.014368e+08,0.0,-3.034789e+08,-2.993947e+08,True
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,-3.097704e+08,0.0,-3.120359e+08,-3.075049e+08,True
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,-1.889359e+07,0.0,-2.129650e+07,-1.649068e+07,True
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-2.722720e+07,0.0,-2.982264e+07,-2.463177e+07,True
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-8.333615e+06,0.0,-1.092905e+07,-5.738178e+06,True



=== Tukey HSD for TotalEnergy_Jperm3 (alpha = 0.05) ===
                                        Multiple Comparison of Means - Tukey HSD, FWER=0.05                                        
             group1                          group2                  meandiff     p-adj        lower             upper       reject
-----------------------------------------------------------------------------------------------------------------------------------
 ./Reference Tests/Reference_T1 3DPrint/Solid_4.is_tens_RawData -83216381981.4013    0.0  -89627537658.905 -76805226303.8977   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_6.is_tens_RawData -83299440069.3537    0.0 -89710595746.8573 -76888284391.8501   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_8.is_tens_RawData -84853474646.9796    0.0  -91966013271.663 -77740936022.2961   True
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_6.is_tens_RawData    -83058087.9523    1.0  -7627044527.3998   7460928351.4951  False
3DPrint/Solid_4.is_

,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,-8.321638e+10,0.0000,-8.962754e+10,-7.680523e+10,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,-8.329944e+10,0.0000,-8.971060e+10,-7.688828e+10,True
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,-8.485347e+10,0.0000,-9.196601e+10,-7.774094e+10,True
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,-8.305809e+07,1.0000,-7.627045e+09,7.460928e+09,False
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-1.637093e+09,0.9382,-9.785529e+09,6.511344e+09,False
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-1.554035e+09,0.9464,-9.702471e+09,6.594402e+09,False



=== Tukey HSD for StrainAtFracture (alpha = 0.05) ===
                           Multiple Comparison of Means - Tukey HSD, FWER=0.05                           
             group1                          group2             meandiff p-adj    lower    upper   reject
---------------------------------------------------------------------------------------------------------
 ./Reference Tests/Reference_T1 3DPrint/Solid_4.is_tens_RawData -84.8004    0.0  -97.6765 -71.9243   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_6.is_tens_RawData -86.4501    0.0  -99.3262  -73.574   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_8.is_tens_RawData -86.3539    0.0 -100.6386 -72.0691   True
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_6.is_tens_RawData  -1.6496 0.9891  -16.8009  13.5016  False
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_8.is_tens_RawData  -1.5534 0.9927  -17.9187  14.8118  False
3DPrint/Solid_6.is_tens_RawData 3DPrint/Solid_8.is_tens_RawData   0.0962    1.0   -16.269  16.461

,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,-84.8004,0.0000,-97.6765,-71.9243,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,-86.4501,0.0000,-99.3262,-73.5740,True
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,-86.3539,0.0000,-100.6386,-72.0691,True
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,-1.6496,0.9891,-16.8009,13.5016,False
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-1.5534,0.9927,-17.9187,14.8118,False
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,0.0962,1.0000,-16.2690,16.4614,False


#GOAL 1
